#### Workshop Description
Understanding the questions posed by instructors and students alike plays an important role in the development of educational technology applications. In this intermediate level workshop, you will learn to apply NLP to one piece of this real-world problem by building a model to predict the type of answer (e.g. entity, description, number, etc.) a question elicits. Specifically, you will learn to:
1. Perform preprocessing, normalization, and exploratory analysis on a question dataset,
2. Identify salient linguistic features of natural language questions, and
3. Experiment with different feature sets and models to predict the answer type.
4. Use powerful pretrained language models to create dense sentence representations and apply deep learning models to text classification.

The concepts will be taught using popular NLP and ML packages like SpaCy, Scikit Learn, and Tensorflow.

This workshop assumes familiarity with Jupyter notebooks and the basics of scientific packages like numPy and sciPy. We also assume some basic knowledge of machine learning and deep learning techniques like CNNs, LSTMs, etc. Reference materials will be provided to gain a better understanding of these techniques for interested attendees.
***

# Feature Extraction & Classification

This notebook is the 2nd of 3 for the Course Hero ODSC workshop "Applications of NLP in EdTech." In this notebook, we will 
1. Extract important features from TREC question-classification text, 
2. Train and evaluate a Maximum Entropy classification model using these features, and 
3. Analyze the impact of each described featureset on the model's performance.

The model we build here is decent, but there is still a lot that can be done to improve it. Treat this a challenge to take what we cover here and expand it on your own to try to get the best possible performance!

**(Spoiler Alert)** In the next module (Module 3), We will also show you how to use deep learning to outperform the model developed here.

In [ ]:
#
# ONLY RUN THIS CELL IF USING COLAB
#
!wget -q https://raw.githubusercontent.com/coursehero/ai-odsc-workshop-2019/master/requirements.txt -O requirements.txt
!pip install -qr requirements.txt


In [ ]:
from typing import Dict, Tuple, List, Union

get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [ ]:
from os.path import exists, join
import pandas as pd
import requests
import en_core_web_md
import spacy
from nltk.corpus import wordnet as wn

nlp = en_core_web_md.load()

#
# Verify that WordNet corpus has been downloaded. If not, download it now.
#
try:
    wn.synsets('dog')
except Exception:
    from nltk import download
    download('wordnet')



### Download Data

First let's download the train and test data from Xin Li, Dan Roth, Learning Question Classifiers. COLING'02, Aug., 2002.
    <https://cogcomp.seas.upenn.edu/Data/QA/QC/">https://cogcomp.seas.upenn.edu/Data/QA/QC/>
    
We will store these data in Pandas DataFrames containing the following columns:
- *question*: The question text
- *processed_question*: The question as a SpaCy Doc object
- *coarse_label*: The coarse-grained label (6 classes)
- *label*: The fine-grained label

Recall that in Module 1, we found that some questions were duplicated. Let's remove those now.

In [ ]:
def format_trec_data(text):
    """
    Convert the whitespace-delimited text format of TREC data to a Pandas
    DataFrame, with the labels processed into fine- and coarse-grained
    alternatives.

    Parameters
    ----------
    text: str
        The full text of the TREC data. Each line consists of the fine-grained
        label (eg "NUM:date") followed by a space and the question text.

    Returns
    -------
    pd.DataFrame
        Pandas DataFrame with three columns: 'question', 'label', and
        'coarse_label'.

    """
    data = [line for line in text.split('\n') if line]
    labels, questions = zip(*[line.split(' ', 1) for line in data])
    coarse_labels = [label.split(':')[0] for label in labels]
    df = pd.DataFrame({"question": questions,
                       "label": labels,
                       "coarse_label": coarse_labels})
    return df

In [ ]:
import pandas as pd
import requests
from os import path, mkdir

train_url = "https://cogcomp.seas.upenn.edu/Data/QA/QC/train_5500.label"
test_url = "https://cogcomp.seas.upenn.edu/Data/QA/QC/TREC_10.label"

data_dir_name = 'data'
try:
    mkdir(data_dir_name)
    print("Directory '{}' created".format(data_dir_name))
except FileExistsError:
    print("Directory '{}' already exists".format(data_dir_name))
    
data = requests.get(train_url).text
train_df = format_trec_data(data)

data = requests.get(test_url).text
test_df = format_trec_data(data)

train_df.to_csv(path.join(data_dir_name, "train.csv"), index=False)
test_df.to_csv(path.join(data_dir_name, "test.csv"), index=False)


#
# Dedupe from python module.
#
train_df = train_df.drop_duplicates("question")
test_df = test_df.drop_duplicates('question')

#
# Process questions with SpaCy
#
train_df['processed_question'] = train_df.question.apply(nlp)
test_df['processed_question'] = test_df.question.apply(nlp)


## Linguistic Features 

In [ ]:
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as sklearn_stopwords
from nltk import ngrams as NLTK_ngrams

### Brief SpaCy Demonstration

In [ ]:
question = "What Polynesian people inhabit New Zealand ?"

#
# Calling `nlp(<text>)` returns a fully-annotated SpaCy Document, 
# sentence and word tokenized. Each of the tokens has a long 
# list of attributes, a sample of which is shown in the display 
# below.
# 
#
annotated_doc = nlp(question)
rows = []
for token in annotated_doc:
    token_attrs = {
        "Text": token.text,
        "POS": token.pos_,
        "TAG": token.tag_,
        "DEP": token.dep_,
        "HEAD": token.head,
        "ENT_TYPE": token.ent_type_,
        "BROWN": token.cluster,
        "SHAPE": token.shape_
    }
    rows.append(token_attrs)
print(pd.DataFrame(rows, columns=["Text", "POS", "TAG", "DEP", "HEAD", "ENT_TYPE", "BROWN", "SHAPE"]))
print('\n')

#
# Additionally, the SpaCy Doc can give us all of the base Noun Phrases
# occurring in the text. A "Base Noun Phrase" is a Noun Phrase that
# doesn't contain any other Noun Phrases in it (note that this is
# distinct from the typical NP in Linguistics).
# 
# For example, the sentence "Which friends of Jerry own the bookstore on the corner"
# has the following "Full" Noun Phrases:
#      1. Which friends of Jerry
#      2. the bookstore on the corner
#
# However, the "Base Noun Phrases" are as follows:
#      1. Which friends
#      2. Jerry
#      3. the bookstore
#      4. the corner
#

print("Base Noun Chunks: {}".format(list(annotated_doc.noun_chunks)))

When we perform machine learning on text data, there are a number of different features that may prove valuable in learning how to distinguish between our classes.

Here, we break them down into "Syntactic Features" and "Semantic Features":

### Syntactic Features: 
Features that reflect the structure of our sentences.


1. **Word ngrams**: These are the length-n word sequences that occur in our data. Often, we add "BOS" and "EOS" tokens to mark the beginning and end of the sentence, respectively. For example, the "bigrams" (n=2) for the sentence "This is an example" would be {"BOS this", "this is", "is an", "an example", "example EOS"}. The order of `n` can be anything, though typically high orders of `n` result in such sparse features that they are no longer very useful. We will experiment with a few different values and ranges of `n`, and ultimately settle on "unigrams" (single words), and "bigrams" (pairs of adjacent words).
  <u>Stopwords and Word ngrams</u>: Often, high-frequency words that serve a mostly functional purpose, eg. "of", "the", "from", are found to be of little help during text classification problems, and are thus removed from the text before ngram features are extracted. We can experiment with different sets of stopwords in our ngram-generation function.


In [ ]:
#
# Explore some different default stopword lists
#

try:
    from nltk.corpus import stopwords
    nltk_stopwords = stopwords.words('english')
except Exception:
    download('stopwords')
    nltk_stopwords = stopwords.words('english')


print("SciKit-Learn stop words")
print(sorted(sklearn_stopwords))
print("\n\n")
print("NLTK stop words")
print(sorted(nltk_stopwords))

print('\n\n')
print(set(nltk_stopwords).difference(sklearn_stopwords))

In [ ]:
def word_ngrams(question: spacy.tokens.doc.Doc, 
                ngram_range: Tuple[int] = (1, 2), 
                stopwords: Union[List[str], None] = None, 
                lower: bool = False) -> Dict[str, int]:
    """
    Generate word n-grams for a piece of text. Use NLTK_ngrams(<List[str]>, n) 
    function to generate n-grams for order `n`.
    
    Parameters
    ----------
    question: spacy.tokens.doc.Doc
        A spacy Doc (ie a string that has been processed via eg `nlp(s)`)
        
    ngram_range: tuple of ints 
        A tuple of min and max n for ngrams.
        
    stopwords: List of str or None 
        List of words to exclude. For present purposes, when `n` > 1, only 
        exclude the ngram if all of its tokens are stopwords.
        
    lower: bool 
        A boolean indicating whether to lowercase the text in the ngrams.

    Returns
    -------
    dict
        Map of feature names to 1.
    """
    ngram_dict = {}    
    stopwords = (set([]) if stopwords is None 
                 else {w.lower() for w in stopwords})
    
    ######################
    tokens = # Write code to create a list of the sequence of tokens
    ######################
    
    min_n, max_n = ngram_range
    for n in range(min_n, max_n + 1):
        if n > 1:
            tokens = ["<BOS>"] + tokens + ["<EOS>"]
        ngrams_for_n = NLTK_ngrams(tokens, n)
        for ng in ngrams_for_n:
            if all(w in stopwords for w in ng):
                continue
            feature_key = "{}gram={}".format(n, " ".join(ng))
            ngram_dict[feature_key] = 1
    return ngram_dict

  2. **Part-of-Speech ngrams**: The length-n sequences of syntactic classes (eg Noun, Verb, Preposition) can allow us to generalize key information about the language used in our data. Since the set of Part-of-Speech tags is significantly smaller than the number of unique words, this feature set is much less sparse than word n-grams and can help us find more general word-level patterns.

In [ ]:
def pos_ngrams(question: spacy.tokens.doc.Doc, 
               ngram_range: Tuple[int] = (1, 2)
              ) -> Dict[str, int]:
    """
    Generate Part-of-speech n-grams for a piece of text. 
    Use NLTK_ngrams(<List[str]>, n) function to generate 
    n-grams for order `n`, and SpaCy's <token>.tag_ for detailed
    Part-of-Speech tags (alternatively you could try 
    <token>.pos_ for simple Part-of-Speech tags from the Universal 
    Part-of-Speech tag set.)
    
    Parameters
    ----------
    question: spacy.tokens.doc.Doc
        A spacy Doc (ie a string that has been processed via eg `nlp(s)`)
        
    ngram_range: tuple of ints 
        A tuple of min and max n for ngrams.
        
    Returns
    -------
    dict
        Map of feature names to 1.
        
    """
    ngram_dict = {}
    
    ###############
    tokens = # Write code to create a list of the sequence of tags
    ###############
    
    min_n, max_n = ngram_range
    for n in range(min_n, max_n + 1):
        if n > 1:
            tokens = ["<BOS>"] + tokens + ["<EOS>"]
        for ng in NLTK_ngrams(tokens, n):
            feature_key = "POS_{}gram={}".format(n, " ".join(ng))
            ngram_dict[feature_key] = 1
    return ngram_dict

  3. **Dependency Triples**: Often, pairs of non-adjacent words, despite being located in different parts of a sentence, have a syntactic relationship. For example, in the two sentences below, the relationship between "What" and "say" can be characterized as one of "direct object" in both, despite their different word orderings. These relationships cannot be captured by n-grams, which only focus on groups of adjacent words. Instead, we can use SpaCy's powerful Universal Dependency parser to extract these relationships, and encode them as features. For details about Universal Dependency relationships, see <https://universaldependencies.org/>

> "What did Joanie say to Chachi?"

> "Joanie said what to Chachi??"

In [ ]:
#
# Demonstrate dependency parses for two sentences with different 
# word orders.
#
doc1 = nlp("What did Joanie say to Chachi?")
doc2 = nlp(u"Joanie said what to Chachi??")


In [ ]:
# spacy.displacy.serve(doc1, style='dep')

In [ ]:
# spacy.displacy.serve(doc2, style='dep')

![dep_nobj-2](images/joanie-chachie-2.png)
![dep_nobj-1](images/joanie-chachie-1.png)

In [ ]:
def dependency_triples(question: spacy.tokens.doc.Doc, 
                       lemmatize: bool = True
                      ) -> Dict[(str, int)]:
    """
    Extract all (head, dependent, relation) dependency-triples for a 
    piece of text. Later, also substitute named entities with their type,
    or substitute either all heads or all dependents with their part of 
    speech.
    
    For each SpaCy token, `<token>.head` gives the text of the head token 
    for its dependency relationship, and `<token>.dep_` gives the relationship 
    type.
    
    To enable lemmatization of the tokens in the dependency triple, use 
    `<token>.lemma_`.
    
    Parameters
    ----------
    question: SpaCy Doc
        A question processed by a SpaCy model.
        
    lemmatize: bool
        Indicates whether or not to use the lemmas instead of the surface 
        token forms.
    
    Returns
    -------
    dict:
        A map of feature names to 1.
        
    """
    deps = {}
    for token in question:
        dependent = token
        
        ##########################
        #
        # Write the code to get the head word 
        # and relation type for each token.
        #
        head = # Your code here
        relation = # Your code here
        ##########################
        
        head, dependent, relation = token.head, token, token.dep_
        if relation == 'ROOT':
            continue
        #
        # If specified, replace the tokens with their lemmas.
        #
        if lemmatize:
            head = head.lemma_
            dependent = dependent.lemma_
        #
        # Otherwise, replace NE tokens with the NE type.
        #
        else:
            head = head.ent_type_ if head.ent_type_ else head
            dependent = dependent.ent_type_ if dependent.ent_type_ else dependent
            
        deps["dependencies#{}-{}-{}".format(head, dependent, relation).lower()] = 1
    return deps


-----------
-----------

**Exercise**: There are a number of interesting variations on the features we define here, as well as many other features that may prove valuable for this task. Use what you know about the data to try implementing your own features (Write feature functions that return a dict mapping <feature_value_name> to 1). For example, you can try replacing one of the words in each dependency triple with its Part-of-Speech (in SpaCy, you can get `<token>.pos_`).

-------
-------

### Semantic Features 
Features that focus on the meaning of word(s) or sentence(s).


  1. **Brown Cluster n-grams**: Brown clusters are a very useful grouping of words into classes based on their distributional information. Thus, words that occur frequently in similar contexts are more likely to be in the same class. This allows us to generalize word n-grams using distributional semantic similarity to reduce feature sparsity. For more details about brown clusters, see [the original paper](https://www.aclweb.org/anthology/J92-4003.pdf). Fortunately, SpaCy's "md" and "lg" pretrained models come with Brown Cluster assignments for all words in the vocabulary. You can access them with `<Token>.cluster`.
  

In [ ]:
import random
from collections import defaultdict
all_word_clusters = defaultdict(list)
for word in nlp.vocab:
    if random.random() < 0.1:
        all_word_clusters[word.cluster].append((word.prob, word.text.lower()))
        
print("{} total clusters".format(len(all_word_clusters)))
print('\n')

for cluster, words in all_word_clusters.items():
    words = sorted(set(words), reverse=True)
    if random.random() < 0.01:
        if len(words) <= 5:
            continue
        else:
            print("{}: {}".format(cluster, random.sample(words, 5)))
        print('\n')

In [ ]:
def brown_cluster_ngrams(question: spacy.tokens.doc.Doc, 
                         ngram_range: Tuple[int] = (1, 2)
                        ) -> Dict[str, int]:
    """
    Generate Brown Cluster id n-grams for a piece of text. 
    
    Parameters
    ----------
    question: spacy.tokens.doc.Doc
        A spacy Doc (ie a string that has been processed via eg `nlp(s)`)
        
    ngram_range: tuple of ints 
        A tuple of min and max n for ngrams.
        
    Returns
    -------
    dict
        Map of feature names to 1.

    """
    ngram_dict = {}
    min_n, max_n = ngram_range
    cluster_id_tokens = [t.cluster for t in question]

    for n in range(min_n, max_n + 1):
        cluster_ngrams = NLTK_ngrams(cluster_id_tokens, n)
        for ng in cluster_ngrams:
            ngram_str = " ".join([str(i) for i in ng])
            feature_key = ("Cluster_{}gram={}".format(n, ngram_str))
            ngram_dict[feature_key] = 1
    return ngram_dict

  2. **Named-Entity Types**: When Named Entities are present in a sentence, use the NE type (ie 'PERSON', 'ORG', etc) as a feature. See <a href="https://spacy.io/api/annotation#named-entities">This list</a> of all of the NE types identified with a pretrained SpaCy model.
  

In [ ]:
def ne_types(question: spacy.tokens.doc.Doc) -> Dict[str, int]:
    """
    Retrives the types of all of the named entities present in a given question.
    
    Parameters
    ----------
    question: SpaCy Doc 
        A question that has been processed via eg `nlp(<question_string>)`
    
    Returns
    -------
    dict
        Map of feature names to 1.

    """
    ents = {}
    for token in question:
        ####################
        ent_type = # Add code here to get the named entity type for each token.
        ####################
        if ent_type is not None:
            ents['NE_type={}'.format(ent_type)] = 1
    return ents

---
#### Aside: Briefly Introducing WordNet with NLTK

In [ ]:
#
# Let's take a look at the different senses of the word "dog", and pick the first one.
#
dog_synsets = wn.synsets('dog')
print(dog_synsets)
print('\n')
dog = dog_synsets[0]

#
# What is this sense's definition?
#
print('Sense Definition:\n-----------------')
print(dog.definition())
print('\n')

#
# Hyponyms: Some types of dog are ___
#
print("(Hyponyms) Some types of dog are: \n---------------------------------")
print(dog.hyponyms())
print('\n')

#
# Hypernyms: A dog is a type of ___
#
print("(Hypernyms) A dog is a type of: \n-------------------------------")
print(dog.hypernyms())
print('\n')

---

  3. **Primary NP-Head Hypernyms**: Many "What" questions ask about a specific Noun Phrase, which we will call "Primary". In the following examples, I've underlined the "Primary" Noun chunk.

> "What <u><em>ocean</em></u> does Mauritania border"</em></p>

> "What is <u>the largest <em>lake</em></u> in the United States?"

> "What is <u> the pig <em>population</em> </u> of the world ?

> "What is <u> the <em>difference</em> </u> between a median and a mean ?"

  In the first two examples, for example, we should be able to detect that they are asking for the same answer type ('LOC'). One helpful technique is to use the Primary Noun Phrase head ("ocean" and "lake") both refer to places/bodies of water/physical entities. One nice way to do identify the similarities between these two NP heads is to use [WordNet](https://wordnet.princeton.edu/) for extraction the semantic relationships between the words.
  In this case, once we identify the head of the Primary Noun Phrase, we would like to generalize it to its <a href="https://en.wikipedia.org/wiki/Hyponymy_and_hypernymy">hypernyms</a>. One easy way to understand this is that a "hyponym" is a "type of" its hypernym. In the previous examples, both "lake" and "ocean" are types of "body of water", "physical entities", etc.</p>

As far as identifying the "primary Noun Phrase" and its "head", we can use some simple, mostly-accurate heuristics (see <a href="https://dl.acm.org/citation.cfm?id=1613835">Question classification using head words and their hypernyms</a> for detailed discussion of this technique and some more advanced approaches that could improve your model). For present purposes, we will use the following simple definition:

_The primary Noun Phrase of "What" (and some "Who") questions is the leftmost Noun chunk after the WH word. Its head is the right-most Noun in that chunk, ignoring Nouns embedded in Prepositional Phrases (The head Noun of "The picture of friend" would be "picture", not "friend")._

In [ ]:
#
# This cell contains all of the functions for generating the set of hypernyms for a 
# "primary" noun head in a "What" question.
#

def get_noun_hypernyms(word: str) -> List[str]:
    """
    Given a word, get the Noun hypernym synsets from WordNet (via nltk).
    
    Parameters
    ----------
    word: str
        The word for which we will return the synsets.
    
    Returns
    -------
    list of str
        All of Noun hypernyms for `word`.
        
    """
    #
    # Get all of the senses of `word` that are Nouns
    #
    noun_synsets = wn.synsets(word, 'n')
    
    #
    # Collect all of the hypernyms for these senses into a list
    # and return the unique hypernyms.
    #
    all_hypernyms = []
    for synset in noun_synsets:
        for hypernym in get_hypernyms_from_synset(synset):
            all_hypernyms.append(hypernym.name().split('.')[0])
    return set(all_hypernyms)


def get_hypernyms_from_synset(synset):
    """
    Given a WordNet Synset, get all of its hypernymic synsets.
    
    Parameters
    ----------
    synset: nltk.corpus.reader.wordnet.Synset
        The synset for which we want the hypernymic synsets.
    
    Returns
    -------
    set of nltk.corpus.reader.wordnet.Synset
        All of the synsets in WordNet that are hypernyms of the 
        provided synset.
        
    """
    hypernyms = set()
    for hypernym in synset.hypernyms():
        #
        # Move "up the tree" to get all of the hypernyms.
        #
        hypernyms |= set(get_hypernyms_from_synset(hypernym))
    return hypernyms | set(synset.hypernyms())


def extract_primary_np_head(
    question: spacy.tokens.doc.Doc
) -> spacy.tokens.token.Token:
    """
    Find the "Primary Noun Head", as defined by our simple heuristic.
    Only try to extract if the provided question is a "What" question.
    
    Parameters
    ----------
    question: SpaCy Doc
        A question processed by a SpaCy model.
    
    Returns
    -------
    SpaCy Token
        The token object corresponding to the head of the primary Noun.
    """
    if question[0].text != 'What':
        return None
    noun_chunks = list(question.noun_chunks)
    if not noun_chunks:
        return None
    
    if len(noun_chunks[0]) > 1:  # is there more than 1 token in the first noun chunk?
        return noun_chunks[0][-1]
    elif len(noun_chunks) >= 2:
        primary_np = noun_chunks[1]
        return primary_np[-1]
        

def primary_np_head_hypernyms(
    question: spacy.tokens.doc.Doc
) -> Dict[str, int]:
    """
    Identify the Primary NP's head token for "What" questions, 
    as defined by our simple heuristic above. Use SpaCy's `<doc>.noun_chunks`
    to get the noun chunks. In this case, we want to find either:
        1) the left-most token in the noun chunk immediately following 'What', or
        2) the Noun
        
    Parameters
    ----------
    question: SpaCy Doc
        A question processed by a SpaCy model.
    
    Returns
    -------
    dict
        Mapping from feature name to 1.
        
    """
    primary_np_head = extract_primary_np_head(question)
    if primary_np_head is None:
        return {}
    else:
        return {"PrimaryNPHeadHypernym={}".format(hyp): 1 
                for hyp in get_noun_hypernyms(primary_np_head.text)}

### Number of Characters
Since we noticed that questions with the fewest characters tended to a certain class, let's also add 

In [ ]:
import math

def length(question: spacy.tokens.doc.Doc) -> Dict[str, int]:
    """
    Get the number of chars to use as a feature. To convert to a binary 
    feature with limited sparsity, calculate the log(base2) of the number 
    of characters, and round the result to the nearest integer.
    
    """
    num_chars = len(str(question))
    binned_length = round(math.log(num_chars, 2))
    return {"log2charcount={}".format(binned_length): 1}

In [ ]:
#
# To combine our features, add each feature function as a key in the dictionary below.
# All functions obligatorily take `question` as their first argument, and optionally
# some keyword arguments. So, the value corresponding to each feature function in the
# dict should be a dict with any keyword arguments.
#

from collections import OrderedDict


all_feature_functions = OrderedDict({
    word_ngrams: {"lower": True}, 
    pos_ngrams: {}, 
    dependency_triples: {'lemmatize': True}, 
    ne_types: {},
    brown_cluster_ngrams: {}, 
    primary_np_head_hypernyms: {}, 
    length: {}
})

In [ ]:
#
# Function to extract all features into a single dict.
#
def create_full_feature_dict(question, feature_funcs_dict):
    feature_dict = {}
    for func, kwargs in feature_funcs_dict.items():
        feature_dict.update(func(question, **kwargs))
    return feature_dict

### 10 Fold Cross-Validation Using Multinomial Logistic Regression
    
   Often, models are evaluated by training on one subset of the data, and predicting/evaluating on a held-out set. To improve on this technique, we can split all of our data into k subsets, and repeat the train/evaluate process k times, each time training on k-1 partitions and evaluating on the remaining partition. This gives us a lower-variance, higher-confidence performance estimate than simply evaluating on a single held out set. This process is known as [cross-validation](https://www.cs.cmu.edu/~schneide/tut5/node42.html)
    
   We will perform 10-fold cross-validation, using the weighted [F1 score](https://en.wikipedia.org/wiki/F1_score), on multiple combinations of featuresets. Specifically, we will start with a single feature (word n-grams), and add featuresets one at a time to later evaluate the impact of each feature on model performance.

   We will use a Maximum Entropy (multinomial Logistic Regression) model as the classifier. The parameter values should provide good performance, but could likely be improved with Grid Search over the hyperparameter space.

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.model_selection import KFold

In [ ]:
f1_scores_dist = {}

In [ ]:
def train_and_evaluate(train_X, train_y, test_X, test_y, print_report=False):
    """
    Train a Maximum Entropy model and calculate the f1 score on a test 
    partition. Optionally, print a classification report.
    """
    #
    # As an exercise, try playing with these hyperparameters.
    # 
    #
    clf = LogisticRegression(C=100., fit_intercept=True, multi_class='auto')
    
    clf.fit(train_X, train_y)
    predictions = clf.predict(test_X)

    if print_report:
        print(classification_report(test_y, predictions))

    f1 = f1_score(test_y, predictions, average='weighted')        
    return f1, clf, predictions

In [ ]:
n_splits = 10
seed = 12345
cv = KFold(n_splits=n_splits, random_state=seed)
folds_indeces = list(cv.split(train_df.index))

In [ ]:
def xval_with_weighted_f1(folds, feature_functions):
    """
    Run cross-validation on `folds` folds, extracting features with the
    feature functions provided.
    
    Parameters
    ----------
    folds: int
        The number of folds.
    
    feature_functions: list of callable
        The feature functions that will be used to generate features for xval.
    
    Returns
    -------
    list of floats
        The weighted f1 score for each fold.
        
    """
    
    feature_vectorizer = DictVectorizer()
    scores = []
    
    feature_dict_func = lambda q: create_full_feature_dict(q, feature_functions)
    
    for i, (train_index, val_index) in enumerate(folds_indeces):
        train_rows = train_df.iloc[train_index]
        train_labels = train_rows.coarse_label
        train_questions = train_rows.processed_question
        train_feature_dicts = train_questions.apply(feature_dict_func)
        train_X = feature_vectorizer.fit_transform(train_feature_dicts)
        
        test_rows = train_df.iloc[val_index]
        test_labels = test_rows.coarse_label
        test_questions = test_rows.processed_question
        test_feature_dicts = test_questions.apply(feature_dict_func)
        test_X = feature_vectorizer.transform(test_feature_dicts)
        
        f1_score, clf, preds = train_and_evaluate(train_X, train_labels, test_X, test_labels)
        scores.append(f1_score)
        
    return scores, (feature_vectorizer, clf, preds, test_labels)

In [ ]:
#
# Calculate the f1-score for each fold of a variety of feature combinations.
#

featureset_score_tracker = {}
current_features_to_use = {}
feature_names = []
for feature_func, kwargs in all_feature_functions.items():
    current_features_to_use[feature_func] = kwargs
    feature_names.append(feature_func.__name__)
    experiment_name = "+".join([''.join([w[0] for w in n.split('_')]) for n in feature_names])
    weighted_f1_dist, (feature_vectorizer, clf, preds, test_labels) = xval_with_weighted_f1(10, 
                                                                               current_features_to_use)
    print("{}: {}".format(experiment_name, weighted_f1_dist))
    featureset_score_tracker[experiment_name] = weighted_f1_dist


### Interpreting Coefficients
We can take the 10th fold of our most recent model (the one that used all of the features together), and which features the model found most impactful in the training data. We do this by using `<model>.coef_[i]`, where `i` is the index of the label of interest. The higher the coefficient value for a feature, the more impact it has on the final prediction. Below we list the top 20 features for each coarse-grained label.

In [ ]:
for i, label in enumerate(('ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM')):
    top_features = sorted(list(zip(feature_vectorizer.get_feature_names(), 
                                   clf.coef_[i])), 
                          key=lambda t: t[1], 
                          reverse=True)[:20]
    print(label)
    for feature_name, weight in top_features:
        print("{}".format(feature_name))
    print('\n')

### Plot Confusion Matrix for 10th Fold

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def plot_confusion_matrix(cm, labels=None):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(cm)
    fig.colorbar(cax)
    if labels:
        ax.set_xticklabels([''] + labels)
        ax.set_yticklabels([''] + labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

labels = sorted(set(test_labels))
cm = confusion_matrix(test_labels, preds, labels=labels)
row_sums = cm.sum(axis=1)
normalized_cm = cm / row_sums[:, np.newaxis]

plot_confusion_matrix(normalized_cm, labels=labels)

### Plot Cross-Validated F1-score for Different Featuresets

In [ ]:
#
# Draw box-and-whisker plots to show the performances over all 10 folds for each 
# featureset combination.
#
labels = sorted(featureset_score_tracker.keys(), key=len)
fig = plt.figure(1, figsize=(8, 5))
ax = fig.add_subplot(111)
data_to_plot = [featureset_score_tracker[l] for l in labels]
ax.boxplot(data_to_plot)
ax.set_xticklabels(labels, rotation=90)
ax.set_ylabel("Weighted F1")
plt.title("Comparison of Some Feature Combinations")
plt.grid(True)
plt.show()

### Train on Entire Training Set and Evaluate on Heldout Data
Now that we have explored how our model performs on different folds of the training data, let's see how well it generalizes to the held-out test set.

In [ ]:
feature_function = lambda q: create_full_feature_dict(q, all_feature_functions)

train_labels = train_df.coarse_label
train_questions = train_df.processed_question
train_feature_dicts = train_questions.apply(feature_function)
train_X = feature_vectorizer.fit_transform(train_feature_dicts)

test_labels = test_df.coarse_label
test_questions = test_df.processed_question
test_feature_dicts = test_questions.apply(feature_function)
test_X = feature_vectorizer.transform(test_feature_dicts)

_, _, preds = train_and_evaluate(train_X, train_labels, test_X, test_labels, print_report=True)

### What's Next?
There is still a lot you can do to improve the model's performance. For instance, you could

1. <u>Hyperparameter Tuning</u>: Experiment with different hyper-parameter settings for this or any other model.

2. <u>Try creating conjunctive features</u>: In addition to adding multiple feature sets as we have done above, another thing you can do is try to find features that, when combined, make a new, more valuable feature. A simple example could be, instead of using one feature to represent a word form and another to represent its part-of-speech, try a conjunctive feature that contains both (eg instead of word unigram "say", and pos unigram "VB", you could make a single feature "say//VB"). 


3. <u> Explore/Combine different models </u>: There are so many ways to model the features we have extracted here. We had some success using a Maximum Entropy model, but you may get some interesting results by using, for instance, a tree-based model instead. Ensemble methods, which use multiple classifiers to improve the final predictions, are also worth exploring here.

##### Resources
- Here is an older but valuable [paper](http://jmlr.csail.mit.edu/papers/volume3/guyon03a/guyon03a.pdf) about the process of feature selection.
- [Maximum Entropy Classifier](http://blog.datumbox.com/machine-learning-tutorial-the-max-entropy-text-classifier/)
- [Ensemble Methods Overview](https://en.wikipedia.org/wiki/Ensemble_learning)
- [Here](https://www.researchgate.net/publication/220637945_From_symbolic_to_sub-symbolic_information_in_question_classification) is a great paper showing how far you can take linguistic feature engineering to achieve very high peformance on this task.


---

In the next module, we will cover how to use representation learning and pre-trained deep learning models as an alternative to the manual feature-engineering we did here. 